# Scaling out numerical computing in Julia
Przemysław Szufel




<a class="anchor" id="toc"></a>
## Table of content

    
1. [Parallelize via Single Instruction Multiple Data (SIMD)](#simd)
2. [Green threading](#green)
3. [Multithreading](#multithreading)
4. [Multi-processing and distributed computing](#multiprocessing)

Before running Jupyter notebook set in Julia number of threads.
This should be done *before* actually running the `notebook()` command.
The number of threads can be also set up in Julia options in Visual Studio code (if this is used to run this notebook).
```
# run this code from Julia console just before starting Jupyter Notebook
ENV["JULIA_NUM_THREADS"]=4
```

In [1]:
println("Number of threads that your Julia is run: ## $(Threads.nthreads())")

Number of threads that your Julia is run: ## 4


In [2]:
using BenchmarkTools, Distributed

<a class="anchor" id="simd"></a>
### Parallelize via Single Instruction Multiple Data (SIMD)
---- [Return to table of contents](#toc) ---



In [3]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot1 (generic function with 1 method)

In [4]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot2 (generic function with 1 method)

In [5]:
x = 100*rand(10000)
y = 100*rand(10000);

@btime dot1($x, $y)
@btime dot2($x, $y)

  6.250 μs (0 allocations: 0 bytes)
  1.190 μs (0 allocations: 0 bytes)


2.5163178516501915e7

In [6]:
res1 =  dot1(x, y)

2.516317851650193e7

In [9]:
res1 =  dot1(x, y)

2.516317851650193e7

In [10]:
res2 =  dot2(x, y)

2.5163178516501915e7

In [7]:
res2 =  dot2(x, y)

2.5163178516501915e7

In [8]:
res1 == res2

false

In [34]:
@show res1 
@show res2

res1 = 2.4943127090471815e7
res2 = 2.4943127090471864e7


2.4943127090471864e7

<a class="anchor" id="green"></a>
### Green threading 
---- [Return to table of contents](#toc) ---


In [11]:
@time sleep(2)

  2.010066 seconds (44 allocations: 1.297 KiB)


In [12]:
@time t = @async sleep(4)

  0.085800 seconds (2.80 k allocations: 201.922 KiB, 99.70% compilation time)


Task (runnable) @0x0000020422383650

In [16]:
function dojob(i)
    val = round(rand(), digits=2)
    sleep(val)   # this could be external computations with I/O
    i, val
end

dojob (generic function with 1 method)

In [19]:
result = Vector{Tuple{Int,Float64}}(undef, 8)

8-element Vector{Tuple{Int64, Float64}}:
 (1, 5.0e-324)
 (1, 5.0e-324)
 (1, 5.0e-324)
 (1, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)
 (0, 0.0)

In [20]:
@time for i=1:8
    result[i] = dojob(i)
end
result

  4.052767 seconds (660 allocations: 41.969 KiB, 0.46% compilation time)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.15)
 (2, 0.11)
 (3, 0.81)
 (4, 0.96)
 (5, 0.15)
 (6, 0.14)
 (7, 0.88)
 (8, 0.78)

In [21]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
   @async result[i] = dojob(i)
end
result

  0.000187 seconds (81 allocations: 7.055 KiB)


8-element Vector{Tuple{Int64, Float64}}:
 (-1, 6.95261695303296e-310)
 (0, 6.95261881366205e-310)
 (2, 6.95261881366205e-310)
 (3, 6.95261881366205e-310)
 (3, 6.9526188136573e-310)
 (2216718762992, 6.95261881365257e-310)
 (140722544826896, 6.95261881365257e-310)
 (140722560313584, 6.95261881365257e-310)

In [22]:
result


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.02)
 (2, 0.92)
 (3, 0.88)
 (4, 0.18)
 (5, 0.99)
 (6, 0.18)
 (7, 0.44)
 (8, 0.2)

In [23]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

  1.003600 seconds (1.49 k allocations: 81.789 KiB, 7.10% compilation time)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.21)
 (2, 0.3)
 (3, 0.11)
 (4, 0.63)
 (5, 0.93)
 (6, 0.68)
 (7, 0.45)
 (8, 0.08)

In [25]:
asyncmap(dojob, 1:8)

8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.17)
 (2, 0.9)
 (3, 0.54)
 (4, 0.5)
 (5, 0.77)
 (6, 0.74)
 (7, 0.56)
 (8, 0.74)

In [24]:
?asyncmap

search: asyncmap asyncmap!



```
asyncmap(f, c...; ntasks=0, batch_size=nothing)
```

Uses multiple concurrent tasks to map `f` over a collection (or multiple equal length collections). For multiple collection arguments, `f` is applied elementwise.

`ntasks` specifies the number of tasks to run concurrently. Depending on the length of the collections, if `ntasks` is unspecified, up to 100 tasks will be used for concurrent mapping.

`ntasks` can also be specified as a zero-arg function. In this case, the number of tasks to run in parallel is checked before processing every element and a new task started if the value of `ntasks_func` is greater than the current number of tasks.

If `batch_size` is specified, the collection is processed in batch mode. `f` must then be a function that must accept a `Vector` of argument tuples and must return a vector of results. The input vector will have a length of `batch_size` or less.

The following examples highlight execution in different tasks by returning the `objectid` of the tasks in which the mapping function is executed.

First, with `ntasks` undefined, each element is processed in a different task.

```
julia> tskoid() = objectid(current_task());

julia> asyncmap(x->tskoid(), 1:5)
5-element Array{UInt64,1}:
 0x6e15e66c75c75853
 0x440f8819a1baa682
 0x9fb3eeadd0c83985
 0xebd3e35fe90d4050
 0x29efc93edce2b961

julia> length(unique(asyncmap(x->tskoid(), 1:5)))
5
```

With `ntasks=2` all elements are processed in 2 tasks.

```
julia> asyncmap(x->tskoid(), 1:5; ntasks=2)
5-element Array{UInt64,1}:
 0x027ab1680df7ae94
 0xa23d2f80cd7cf157
 0x027ab1680df7ae94
 0xa23d2f80cd7cf157
 0x027ab1680df7ae94

julia> length(unique(asyncmap(x->tskoid(), 1:5; ntasks=2)))
2
```

With `batch_size` defined, the mapping function needs to be changed to accept an array of argument tuples and return an array of results. `map` is used in the modified mapping function to achieve this.

```
julia> batch_func(input) = map(x->string("args_tuple: ", x, ", element_val: ", x[1], ", task: ", tskoid()), input)
batch_func (generic function with 1 method)

julia> asyncmap(batch_func, 1:5; ntasks=2, batch_size=2)
5-element Array{String,1}:
 "args_tuple: (1,), element_val: 1, task: 9118321258196414413"
 "args_tuple: (2,), element_val: 2, task: 4904288162898683522"
 "args_tuple: (3,), element_val: 3, task: 9118321258196414413"
 "args_tuple: (4,), element_val: 4, task: 4904288162898683522"
 "args_tuple: (5,), element_val: 5, task: 9118321258196414413"
```


#### Programming a simple web server
You should be able to connect using the address <a href="http://localhost:9992/3+4" target="about:blank">http://localhost:9992/3+4</a>

To stop web server click <a href="http://localhost:9992/stopme" target="about:blank">http://localhost:9992/stopme</a>

In [26]:
using Sockets
println("Starting the web server...")
server = Sockets.listen(9992)

Starting the web server...


Sockets.TCPServer(Base.Libc.WindowsRawSocket(0x0000000000000458) active)

In [27]:
@async begin
    contt = Ref(true)
    while contt[]
        sock = Sockets.accept(server)
        @async begin
            data = readline(sock)
            print("Got request:\n", data, "\n")
            cmd = split(data, " ")[2][2:end]
            println(sock, "\nHTTP/1.1 200 OK\nContent-Type: text/html\n")
            contt[] = contt[] && (!occursin("stopme", data))
            if contt[]
                 println(sock, string("<html><body>", cmd, "=", 
                     eval(Meta.parse(cmd)), "</body></html>"))
            else
                println(sock,"<html><body>stopping</body></html>")
            end
            close(sock)
        end
    end
    println("Handling requests stopped")
end

Task (runnable) @0x0000020459fa9180

Got request:
GET /2+2 HTTP/1.1
Got request:
GET /favicon.ico HTTP/1.1
Got request:
GET /2*2 HTTP/1.1
Got request:
GET /favicon.ico HTTP/1.1


<a class="anchor" id="multithreading"></a>
### Multithreading
---- [Return to table of contents](#toc) ---


In [29]:
Threads.nthreads()

4

In [30]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end

ssum (generic function with 1 method)

In [39]:
@macroexpand Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end

quote
    #= threadingconstructs.jl:180 =#
    local var"#170#threadsfor_fun"
    #= threadingconstructs.jl:181 =#
    let var"#169#range" = 1:c
        #= threadingconstructs.jl:182 =#
        function var"#170#threadsfor_fun"(var"#181#tid" = 1; onethread = false)
            #= threadingconstructs.jl:182 =#
            #= threadingconstructs.jl:183 =#
            var"#174#r" = var"#169#range"
            #= threadingconstructs.jl:184 =#
            var"#175#lenr" = Base.Threads.length(var"#174#r")
            #= threadingconstructs.jl:186 =#
            if onethread
                #= threadingconstructs.jl:187 =#
                var"#181#tid" = 1
                #= threadingconstructs.jl:188 =#
                (var"#176#len", var"#177#rem") = (var"#175#lenr", 0)
            else
                #= threadingconstructs.jl:190 =#
                (var"#176#len", var"#177#rem") = Base.Threads.divrem(var"#175#lenr", Base.Threads.threadpoolsize())
            end
            #= threadingco

In [31]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end


tsum (generic function with 1 method)

In [32]:
x = rand(1000,10000);

In [37]:
@time ssum(x)
@time ssum(x);

  0.028556 seconds (2 allocations: 78.172 KiB)
  0.019815 seconds (2 allocations: 78.172 KiB)


In [38]:
@time tsum(x)
@time tsum(x);

  0.005878 seconds (25 allocations: 80.781 KiB)
  0.004839 seconds (25 allocations: 80.930 KiB)


#### Locking mechanism for threads

In [40]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^6
        x += 1
    end
    return x
end


f_bad (generic function with 1 method)

In [83]:
f_bad()

308113

In [85]:
function f_add()
    x = 0 
    for i in 1:10^6
        x += 1
    end
    x
end
@btime f_add()
    

  2.200 ns (0 allocations: 0 bytes)


1000000

In [87]:
@code_native f_add()

	.text
	.file	"f_add"
	.globl	julia_f_add_2804                # -- Begin function julia_f_add_2804
	.p2align	4, 0x90
	.type	julia_f_add_2804,@function
julia_f_add_2804:                       # @julia_f_add_2804
; ┌ @ In[85]:1 within `f_add`
	.cfi_startproc
# %bb.0:                                # %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
; │ @ In[85]:6 within `f_add`
	mov	eax, 1000000
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_f_add_2804, .Lfunc_end0-julia_f_add_2804
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


In [88]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^6
        Threads.atomic_add!(x, 1)
    end
    return x[]
end
f_atomic()

1000000

In [ ]:
u

In [92]:
function f_easy()
    result = zeros(Int, Threads.nthreads())
    Threads.@threads for i in 1:10^6
        result[Threads.threadid()] += 1
    end
    return sum(result)
end

f_easy (generic function with 1 method)

In [89]:
function f_spin()
    l = Threads.SpinLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end

function f_reentrant()
    l = ReentrantLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end


f_reentrant (generic function with 1 method)

In [93]:
using DataFrames
stats = DataFrame()
for f in [f_bad, f_atomic, f_spin, f_reentrant]
    for i = 1:2
        value, elapsedtime  = @timed f()
        push!(stats, (f=string(f),i=i, value=value, timems=elapsedtime*1000))
    end
end
println(stats)


8×4 DataFrame
 Row │ f            i      value    timems   
     │ String       Int64  Int64    Float64  
─────┼───────────────────────────────────────
   1 │ f_bad            1   250000   53.5209
   2 │ f_bad            2   502149   54.0777
   3 │ f_atomic         1  1000000   24.1497
   4 │ f_atomic         2  1000000   24.647
   5 │ f_spin           1  1000000  752.861
   6 │ f_spin           2  1000000  473.942
   7 │ f_reentrant      1  1000000  853.451
   8 │ f_reentrant      2  1000000  730.869


<a class="anchor" id="multiprocessing"></a>
### Multi-processing and distributed computing
---- [Return to table of contents](#toc) ---


In [94]:
using Distributed

This code adds 4 workers (and avoids adding more)

In [95]:
addprocs(max(0, 5-nprocs()));

In [96]:
workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [97]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()


  1.113896 seconds (20.00 k allocations: 763.397 MiB, 19.71% gc time)
  1.093252 seconds (20.00 k allocations: 763.397 MiB, 13.90% gc time)


4999.6947155994

In [98]:
using Distributed
 
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        # the last line will be aggregated
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()


  6.293818 seconds (465.16 k allocations: 31.234 MiB, 20.16% compilation time)
  0.655145 seconds (437 allocations: 24.734 KiB)


5000.301076551798

In [63]:
workers()'

1×4 adjoint(::Vector{Int64}) with eltype Int64:
 2  3  4  5

In [64]:
fetch(@spawnat 3 4+3)

7

In [65]:
function myf() 
    println("I am on worker ", myid())
    rand()
end
myf()

I am on worker 1


0.21248315596630618

In [66]:
a = nothing
try 
    fetch(@spawnat 4 myf())
catch e
    println(e)
end

RemoteException(4, CapturedException(UndefVarError(Symbol("#myf")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_global_from_main at clusterserialize.jl:160, 1), (#5 at clusterserialize.jl:72 [inlined], 1), (foreach at abstractarray.jl:3086, 1), (deserialize at clusterserialize.jl:72, 1), (handle_deserialize at Serialization.jl:960, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:871, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:887 [inlined], 1), (invokelatest at essentials.jl:884 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_s

In [67]:
@everywhere function myf() 
    println("I am on worker ", myid())
    rand()
end
fetch(@spawnat 4 myf())

      From worker 4:	I am on worker 4


0.7642915859931477

#### A typical pattern for setting an intial state across workers

In [100]:
using Distributed
@everywhere using Pkg
@everywhere Pkg.activate("../.")
@everywhere using Distributed, Random, DataFrames

@everywhere function calc(x, y)
    2x + y
end

@everywhere function init_worker()    
   Random.seed!(myid())
    # reading initial data from files or other actions
end

@sync for wid in workers()
    @async fetch(@spawnat wid init_worker())
end


      From worker 3:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julia_EC_EconomicDep`
      From worker 2:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julia_EC_EconomicDep`

  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julia_EC_EconomicDep`



      From worker 5:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julia_EC_EconomicDep`
      From worker 4:	  Activating project at `C:\AAABIBLIOTEKA\EuropeanCommission\2024_Julia_EC_EconomicDep`


Typically results are collected to a `DataFrame`

In [101]:
data = @distributed (append!) for (i, j) = vec(collect(Iterators.product(1:4, 1:3)))
    a = rand(1:499)
    b = rand(1:9)*1000
    c = calc(a, b)
    DataFrame(;i,j,a,b,c,procid = myid())
end

Row,i,j,a,b,c,procid
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,20,9000,9040,2
2,2,1,366,1000,1732,2
3,3,1,182,1000,1364,2
4,4,1,185,2000,2370,3
5,1,2,154,7000,7308,3
6,2,2,346,8000,8692,3
7,3,2,454,3000,3908,4
8,4,2,416,2000,2832,4
9,1,3,265,3000,3530,4


In [107]:
pmap(x-> (;red=myid()*x), 10_000:10_020) |> DataFrame

Row,red
,Int64
1,40000
2,30003
3,50010
4,20006
5,40016
6,30015
7,50030
8,20014
9,40032
